# Autonomous Resume RAG Agent Lab: Robo7 Goes Pro! 🤖📄

Welcome to the Autonomous Resume RAG Agent Lab! In this notebook, you'll build a super-smart agent that answers interview questions using your resume—and even sends you an SMS if it can't find the answer. Robo7 is here to help, and he's ready to show off his multitasking skills!

<table><tr>
<td><img src="langchain/images/robo7.png" alt="Robo7 - Autonomous RAG Agent" width="120" /></td>
<td style="vertical-align:top; padding-left:20px;">
<b>Robo7 says:</b><br>
<i>"I can search, answer, and even text you if I get stuck. But I still can't make coffee!"</i><br>
</td>
</tr></table>

Let's get started and see how autonomous agents make RAG even cooler!

## Lab Objectives

- Load your resume PDF and chunk it for retrieval.
- Build custom tools: one for searching your resume, one for sending SMS.
- Create an autonomous agent using LangChain's latest tool-calling syntax.
- Ask the agent interview questions and see how it responds!
- If the answer isn't found, Robo7 will send you an SMS with the question.

---

### Step-by-Step Overview
1. **Setup & Imports**: Load all required libraries and environment variables.
2. **Resume Loader & Vector Store**: Ingest and chunk your resume, then create a FAISS vector store for retrieval.
3. **Custom Tools**: Implement `resume_search_tool` and `send_sms_tool`.
4. **Agent Creation**: Build an autonomous agent with LangChain's latest syntax.
5. **Demo Q&A**: Ask a question and watch Robo7 answer (or send an SMS if he can't!).

Ready? Let's code!

In [3]:
# Step 1: Setup & Imports
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from twilio.rest import Client

# Load environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_FROM_NUMBER = os.getenv("TWILIO_FROM_NUMBER")
TWILIO_TO_NUMBER = os.getenv("TWILIO_TO_NUMBER")
RESUME_PATH = os.getenv("RESUME_PATH", "my_resume.pdf")  # Update if needed

# Check config
print("OpenAI Key Loaded:", bool(OPENAI_API_KEY))
print("Twilio Config Loaded:", bool(TWILIO_ACCOUNT_SID and TWILIO_AUTH_TOKEN))
print("Resume Path:", RESUME_PATH)

OpenAI Key Loaded: True
Twilio Config Loaded: True
Resume Path: my_resume.pdf


In [9]:
# Step 2: Load & Chunk Resume, Create Vector Store
loader = PyPDFLoader(RESUME_PATH)
documents = loader.load()

# Chunk resume into smaller pieces for retrieval
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Create FAISS vector store
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(docs, embeddings)

print(f"Loaded {len(docs)} resume chunks into FAISS vector store.")

Loaded 128 resume chunks into FAISS vector store.


In [ ]:
# Step 3: Define Custom Tools

def resume_search_tool(query: str) -> str:
    """Searches the resume for relevant information."""
    results = vectorstore.similarity_search(query, k=3)
    if results:
        return "\n---\n".join([doc.page_content for doc in results])
    return "No relevant information found in the resume."


def send_email_tool(message: str) -> str:
    """Sends an email using Gmail SMTP."""
    import smtplib
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart

    GMAIL_USER = os.getenv("GMAIL_USER")
    GMAIL_PASS = os.getenv("GMAIL_PASS")
    TO_EMAIL = os.getenv("TO_EMAIL")
    if not all([GMAIL_USER, GMAIL_PASS, TO_EMAIL]):
        return "Missing Gmail credentials or recipient email in environment variables."

    try:
        msg = MIMEMultipart()
        msg['From'] = GMAIL_USER
        msg['To'] = TO_EMAIL
        msg['Subject'] = "Robo7: Interview Question Not Found"
        msg.attach(MIMEText(message, 'plain'))

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(GMAIL_USER, GMAIL_PASS)
        server.sendmail(GMAIL_USER, TO_EMAIL, msg.as_string())
        server.quit()
        return f"Email sent to {TO_EMAIL}!"
    except Exception as e:
        return f"Failed to send email: {e}"

# Create LangChain tools
resume_tool = Tool(
    name="ResumeSearch",
    description="Searches the resume for answers to interview questions.",
    func=resume_search_tool
)
email_tool = Tool(
    name="SendEmail",
    description="Sends an email if the answer is not found in the resume.",
    func=send_email_tool
)
tools = [resume_tool, email_tool]

In [11]:
# Step 4: Create Autonomous Agent

prompt = PromptTemplate.from_template(
    """
    You are Robo7, an autonomous interview assistant. Use the ResumeSearch tool to answer questions from the resume. If you can't find the answer, use the SendSMS tool to notify the user with the question.
    
    Question: {input}
    {agent_scratchpad}
    """
)

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools)

print("Robo7 agent is ready!")

Robo7 agent is ready!


## Demo Time: Ask Robo7 an Interview Question!

Type your interview question below and run the cell. Robo7 will search your resume and answer. If he can't find the answer, he'll send you an email with the question!

> **Example questions:**
> - What programming languages do I know?
> - What is my experience with project management?
> - What certifications do I have?

---

Ready to see Robo7 in action? 🚀

In [ ]:
# Step 5: Ask Robo7 a Question!

# Type your interview question here:
question = "Do you have a patent?"

response = agent_executor.invoke({"input": question})
print("Robo7's answer:")
print(response["output"])

# If Robo7 can't find the answer, he'll send you an email with the question!

---

# 🎉 Lab Complete! Robo7 is now your autonomous interview assistant! 🤖

<img src="langchain/images/robo7.png" alt="Robo7 - Autonomous RAG Agent" width="100" />

**Robo7 says:**
> "Great job! Now I can answer your questions and text you if I get stuck. If only I could negotiate your salary too!"

---

Feel free to experiment with more questions, tweak the agent, or add new tools. The future of autonomous agents is in your hands!